In [1]:
import os
import sys
import pdb
import time
import glob
import functools
import multiprocessing
import numpy as np
import soundfile as sf
import scipy.interpolate
import scipy.io
import scipy.signal
import h5py
import tqdm
import soxr
import importlib
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_figures
import util_stimuli
import util_misc

import simulator
importlib.reload(simulator)


<module 'simulator' from '/rdma/vast-rdma/vast/mcdermott/msaddler/python-packages/virtual_acoustic_room/simulator.py'>

In [ ]:
hrtf_locs, hrtf_firs, hrtf_sr = simulator.load_kemar_hrtfs(npz_filename='kemar_hrtfs/hrtfs.npz')

IDX_nonzero_azim = hrtf_locs[:, 1] != 0
opposite_hrtf_locs = hrtf_locs[IDX_nonzero_azim, :]
opposite_hrtf_locs[:, 1] = -1 * opposite_hrtf_locs[:, 1] # Multiply azimuth by negative one
opposite_hrtf_firs = np.flip(hrtf_firs[IDX_nonzero_azim, :, :], axis=2) # Switch left/right channel
hrtf_locs = np.concatenate([opposite_hrtf_locs, hrtf_locs], axis=0)
hrtf_firs = np.concatenate([opposite_hrtf_firs, hrtf_firs], axis=0)
hrtf_locs.shape, hrtf_firs.shape

"""
Check kemar_hrtf locations (azimuths and elevations)
"""
fig, ax = plt.subplots()
x = hrtf_locs[:, 1]
y = hrtf_locs[:, 2]
ax.plot(x, y, 'k.')
ax = util_figures.format_axes(
    ax,
    str_xlabel='Azimuth (°)',
    str_ylabel='Elevation (°)')
plt.show()


In [ ]:
"""
Run this cell to start the MATLAB engine for Python and import `simulator_matlab`
"""
import matlab.engine
import simulator_matlab
importlib.reload(simulator_matlab)

eng = matlab.engine.start_matlab();
if 'src/' not in eng.path():
    eng.addpath('src/');


In [ ]:
"""
Quit MATLAB engine
"""
eng.quit()


In [ ]:
"""
DEBUGGING: CLOSELY COMPARE MATLAB AND PYTHON SIMULATORS UNDER DIFFERENT CONDITIONS

NOTES:
- Resulting BRIRs are sometimes offset by one or two samples
    - May be due to an off-by-one indexing error when porting MATLAB to Python
    - May be due to numerical precision differences
"""
room_materials = [26]*6#[1, 1, 1, 1, 15, 16]
room_dim_xyz = [7, 8, 4]
head_pos_xyz = [3.6, 1.4, 2.0]
head_azim = -49
src_azim = 145
src_elev = 20
src_dist = 3.4
buffer = 0.5

c = 344.5
buffer = 0
sr = 44100
dur = 0.5
use_hrtf_symmetry = True
use_highpass = True
use_jitter = False
incorporate_lead_zeros = True
use_log_distance = True
hrtf_locs = None
hrtf_firs = None

importlib.reload(simulator)

np.random.seed(0)
brir_python = simulator.get_brir(
    room_materials=room_materials,
    room_dim_xyz=room_dim_xyz,
    head_pos_xyz=head_pos_xyz,
    head_azim=head_azim,
    src_azim=src_azim,
    src_elev=src_elev,
    src_dist=src_dist,
    buffer=buffer,
    sr=sr,
    c=c,
    dur=dur,
    hrtf_locs=hrtf_locs,
    hrtf_firs=hrtf_firs,
    use_hrtf_symmetry=use_hrtf_symmetry,
    use_log_distance=use_log_distance,
    use_jitter=use_jitter,
    use_highpass=use_highpass,
    incorporate_lead_zeros=incorporate_lead_zeros,
    processes=8,
    strict=True,
    verbose=2)


In [ ]:
importlib.reload(simulator_matlab)

np.random.seed(0)
brir_matlab = simulator_matlab.get_brir(
    room_materials=room_materials,
    room_dim_xyz=room_dim_xyz,
    head_pos_xyz=head_pos_xyz,
    head_azim=head_azim,
    src_azim=src_azim,
    src_elev=src_elev,
    src_dist=src_dist,
    buffer=buffer,
    sr=sr,
    c=c,
    dur=dur,
    use_hrtf_symmetry=use_hrtf_symmetry,
    use_log_distance=use_log_distance,
    use_jitter=use_jitter,
    use_highpass=use_highpass,
    incorporate_lead_zeros=incorporate_lead_zeros,
    eng=eng)


In [ ]:
fig, ax_arr = plt.subplots(figsize=(16, 8), nrows=2, ncols=1)

offset_brir_python = brir_python[1:]
offset_brir_matlab = brir_matlab[:-1]

ax_arr[0].plot(offset_brir_python[:, 0], 'b')
ax_arr[0].plot(offset_brir_python[:, 1], 'r')
ax_arr[0].plot(offset_brir_matlab[:, 0], 'c', ls=':')
ax_arr[0].plot(offset_brir_matlab[:, 1], 'm', ls=':')
ax_arr[1].plot(offset_brir_python[:, 0] - offset_brir_matlab[:, 0], 'b')
ax_arr[1].plot(offset_brir_python[:, 1] - offset_brir_matlab[:, 1], 'r')

for ax in ax_arr:
    util_figures.format_axes(
        ax,
        xlimits=[0, 500],
#         ylimits=[-0.01, 0.01]
    )
plt.show()


In [ ]:
importlib.reload(simulator)
simulator.sample_head_parameters(
        room_dim_xyz=[3, 3, 2.2],
        buffer=1.45,
        range_src_elev=[-40, 60],
        range_head_azim=[0, 90],
        range_head_z=[-np.inf, 2],
        verbose=True)


In [36]:
importlib.reload(simulator)

list_df_room = []
for index_room in range(200):
    np.random.seed(index_room)
    room_parameters = simulator.sample_room_parameters(
        verbose=False)
    head_parameters = simulator.sample_head_parameters(
        room_dim_xyz=room_parameters['room_dim_xyz'],
        verbose=False)
    d = {'index_room': index_room}
    d.update(room_parameters)
    d.update(head_parameters)
    list_df_room.append(d)
df_room = pd.DataFrame(list_df_room).sort_index(axis=1)
print(f"Sampled {len(df_room)} rooms and listener positions")


def func_to_parallelize(index_room):
    dfi_room = df_room.iloc[index_room]
    list_d = []
    for src_elev in range(-20, 61, 10):
        for src_azim in range(0, 360, 5):
            for fixed_src_dist in [1.4, None]:
                if fixed_src_dist is not None:
                    src_dist = fixed_src_dist
                else:
                    src_dist_to_wall = simulator.distance_to_wall(
                        room_dim_xyz=dfi_room.room_dim_xyz,
                        head_pos_xyz=dfi_room.head_pos_xyz,
                        head_azim=dfi_room.head_azim,
                        src_azim=src_azim,
                        src_elev=src_elev)
                    min_src_dist = 1.0
                    max_src_dist = src_dist_to_wall - 0.1
                    assert max_src_dist >= min_src_dist
                    src_dist = np.random.uniform(low=1.0, high=src_dist_to_wall)
                src_pos_xyz = np.array([
                    src_dist * np.cos(np.deg2rad(src_elev)) * np.cos(np.deg2rad(src_azim + dfi_room.head_azim)) + dfi_room.head_pos_xyz[0],
                    src_dist * np.cos(np.deg2rad(src_elev)) * np.sin(np.deg2rad(src_azim + dfi_room.head_azim)) + dfi_room.head_pos_xyz[1],
                    src_dist * np.sin(np.deg2rad(src_elev)) + dfi_room.head_pos_xyz[2],
                ])
                assert simulator.is_valid_position(dfi_room.head_pos_xyz, dfi_room.room_dim_xyz, buffer=0), "Invalid head position"
                if not simulator.is_valid_position(src_pos_xyz, dfi_room.room_dim_xyz, buffer=0):
                    print("TOO CLOSE TO WALL!", src_azim, src_elev, src_dist)
                    print('srce', ['{:06.3f}'.format(_) for _ in src_pos_xyz])
                    print('Room', ['{:06.3f}'.format(_) for _ in dfi_room.room_dim_xyz])
                    print('Head', ['{:06.3f}'.format(_) for _ in dfi_room.head_pos_xyz])
                d = {
                    'index_room': dfi_room.index_room,
                    'room_materials': dfi_room.room_materials,
                    'room_dim_xyz': dfi_room.room_dim_xyz,
                    'head_pos_xyz': dfi_room.head_pos_xyz,
                    'head_azim': dfi_room.head_azim,
                    'src_azim': src_azim,
                    'src_elev': src_elev,
                    'src_dist': src_dist,
                    'buffer': 0,
                    'sr': 44100,
                    'c': 344.5,
                    'dur': 0.5,
                    'use_hrtf_symmetry': True,
                    'use_log_distance': False,
                    'use_jitter': True,
                    'use_highpass': True,
                    'incorporate_lead_zeros': True,
                }
                list_d.append(d)
    return pd.DataFrame(list_d).sort_index(axis=1)


with multiprocessing.Pool(20) as p:
    list_df_brir = p.map(func_to_parallelize, range(len(df_room)))
df_brir = pd.concat(list_df_brir).reset_index(drop=True)

df_brir


Sampled 200 rooms and listener positions


,buffer,c,dur,head_azim,head_pos_xyz,incorporate_lead_zeros,index_room,room_dim_xyz,room_materials,sr,src_azim,src_dist,src_elev,use_highpass,use_hrtf_symmetry,use_jitter,use_log_distance
0,0,344.5,0.5,30.365654,"[7.531689501460805, 5.031981696045566, 1.82493...",True,0,"[15.570789053085491, 12.019450985893185, 5.020...","[4, 6, 6, 6, 2, 17]",44100,0,1.400000,-20,True,True,True,False
1,0,344.5,0.5,30.365654,"[7.531689501460805, 5.031981696045566, 1.82493...",True,0,"[15.570789053085491, 12.019450985893185, 5.020...","[4, 6, 6, 6, 2, 17]",44100,0,4.866202,-20,True,True,True,False
2,0,344.5,0.5,30.365654,"[7.531689501460805, 5.031981696045566, 1.82493...",True,0,"[15.570789053085491, 12.019450985893185, 5.020...","[4, 6, 6, 6, 2, 17]",44100,5,1.400000,-20,True,True,True,False
3,0,344.5,0.5,30.365654,"[7.531689501460805, 5.031981696045566, 1.82493...",True,0,"[15.570789053085491, 12.019450985893185, 5.020...","[4, 6, 6, 6, 2, 17]",44100,5,2.926412,-20,True,True,True,False
4,0,344.5,0.5,30.365654,"[7.531689501460805, 5.031981696045566, 1.82493...",True,0,"[15.570789053085491, 12.019450985893185, 5.020...","[4, 6, 6, 6, 2, 17]",44100,10,1.400000,-20,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259195,0,344.5,0.5,9.763437,"[22.93385579962595, 4.141028048603807, 1.05582...",True,199,"[28.784622584640655, 7.534005845466415, 7.8980...","[7, 5, 7, 7, 1, 16]",44100,345,6.197610,60,True,True,True,False
259196,0,344.5,0.5,9.763437,"[22.93385579962595, 4.141028048603807, 1.05582...",True,199,"[28.784622584640655, 7.534005845466415, 7.8980...","[7, 5, 7, 7, 1, 16]",44100,350,1.400000,60,True,True,True,False
259197,0,344.5,0.5,9.763437,"[22.93385579962595, 4.141028048603807, 1.05582...",True,199,"[28.784622584640655, 7.534005845466415, 7.8980...","[7, 5, 7, 7, 1, 16]",44100,350,5.964705,60,True,True,True,False
259198,0,344.5,0.5,9.763437,"[22.93385579962595, 4.141028048603807, 1.05582...",True,199,"[28.784622584640655, 7.534005845466415, 7.8980...","[7, 5, 7, 7, 1, 16]",44100,355,1.400000,60,True,True,True,False


In [44]:
df_room[df_room.is_outdoor == 0]


,head_azim,head_pos_xyz,index_room,is_outdoor,material_x0,material_x1,material_y0,material_y1,material_z0,material_z1,room_dim_xyz,room_materials
0,30.365654,"[7.531689501460805, 5.031981696045566, 1.82493...",0,0,Marble,Plywood,Plywood,Plywood,"Concrete, painted","Acoustic tiles, 0.625"", 16"" below ceiling","[15.570789053085491, 12.019450985893185, 5.020...","[4, 6, 6, 6, 2, 17]"
1,41.148433,"[4.40144279423138, 1.5038636726308217, 1.90811...",1,0,"Fiberglass wall treatment, 7 in","Fiberglass wall treatment, 7 in","Fiberglass wall treatment, 7 in","Fiberglass wall treatment, 7 in",Carpet on foam rubber padding,Window Glass,"[15.755991874283916, 3.0007901772933865, 3.477...","[10, 10, 10, 10, 15, 3]"
2,26.731652,"[1.5658976949844203, 2.8191764717039494, 1.967...",2,0,"Concrete, painted",Brick,Brick,"Concrete, painted",Brick,"Plaster, gypsum, or lime on lath","[3.1845457067025356, 10.63613234488006, 4.2528...","[2, 1, 1, 2, 1, 16]"
3,53.177654,"[9.512605209074819, 2.2748048286932714, 1.6542...",3,0,Marble,Window Glass,"Concrete, painted","Concrete, painted",Linoleum,"Acoustic tiles, 0.625"", 16"" below ceiling","[15.320363767796005, 5.861732474282765, 4.7679...","[4, 3, 2, 2, 13, 17]"
4,73.075297,"[4.225128193953195, 16.717149532207557, 1.3504...",4,0,"Fiberglass wall treatment, 1 in",Marble,Plywood,Plywood,Marble,Plywood,"[10.576780801136788, 28.171217370977573, 6.493...","[9, 4, 6, 6, 4, 6]"
...,...,...,...,...,...,...,...,...,...,...,...,...
195,31.714126,"[1.791533188874179, 4.282679489757011, 1.06347...",195,0,Plywood,Plywood,Plywood,Plywood,Linoleum,"Acoustic tiles, 0.5"" cemented to ceiling","[3.7609084199249208, 6.753508048038817, 5.2699...","[6, 6, 6, 6, 13, 19]"
196,66.572224,"[7.890632630937065, 1.8143972720410209, 1.1084...",196,0,"Concrete block, coarse","Concrete, painted","Concrete block, coarse","Concrete block, coarse",Brick,"Acoustic tiles, 0.625"", 16"" below ceiling","[11.352634228919959, 3.363626924982814, 2.8853...","[7, 2, 7, 7, 1, 17]"
197,63.295934,"[4.08046079920861, 15.121797511795911, 1.16982...",197,0,"Concrete, painted",Window Glass,"Concrete, painted","Concrete, painted",Carpet on foam rubber padding,"Highly absorptive panels, 1"", 16"" below ceiling","[18.273601179271648, 28.404233207184056, 7.045...","[2, 3, 2, 2, 15, 20]"
198,28.211150,"[8.463946607237117, 4.509868086433982, 1.06442...",198,0,"Fiberglass wall treatment, 1 in",Plaster on Concrete,Brick,Brick,Linoleum,Window Glass,"[27.2917426268691, 6.092649983354945, 2.410730...","[9, 5, 1, 1, 13, 3]"


In [ ]:
importlib.reload(simulator);

for _ in range(100):
    np.random.seed(_)
    room_parameters = simulator.sample_room_parameters(
        verbose=False)
    head_parameters = simulator.sample_head_parameters(
        room_dim_xyz=room_parameters['room_dim_xyz'],
        verbose=False)
    room_dim_xyz = room_parameters['room_dim_xyz']
    head_azim = head_parameters['head_azim']
    head_azim = 0
    head_pos_xyz = head_parameters['head_pos_xyz']
    buffer = 0
    for src_azim in range(0, 360, 15):
        for src_elev in range(-60, 61, 5):
            src_dist = simulator.distance_to_wall(
                room_dim_xyz,
                head_pos_xyz,
                head_azim,
                src_azim,
                src_elev)
            src_pos_xyz = np.array([
                src_dist * np.cos(np.deg2rad(src_elev)) * np.cos(np.deg2rad(src_azim + head_azim)) + head_pos_xyz[0],
                src_dist * np.cos(np.deg2rad(src_elev)) * np.sin(np.deg2rad(src_azim + head_azim)) + head_pos_xyz[1],
                src_dist * np.sin(np.deg2rad(src_elev)) + head_pos_xyz[2],
            ])
            assert simulator.is_valid_position(head_pos_xyz, room_dim_xyz, buffer=buffer), "Invalid head position"
            if (np.sum(np.isclose(src_pos_xyz, 0)) == 1) or (np.sum(np.isclose(src_pos_xyz, room_dim_xyz)) == 1):
                pass
            else:
                print(src_azim, src_elev, room_dim_xyz, src_pos_xyz)


In [ ]:
importlib.reload(simulator)

for _ in range(5):
    room_parameters = simulator.sample_room_parameters(verbose=False)
    head_parameters = simulator.sample_head_parameters(room_dim_xyz=room_parameters['room_dim_xyz'], verbose=False)
    room_dim_xyz = room_parameters['room_dim_xyz']
    head_azim = head_parameters['head_azim']
    head_pos_xyz = head_parameters['head_pos_xyz']
    
    fig, ax = plt.subplots()
    rect = matplotlib.patches.Rectangle((0, 0), room_dim_xyz[0], room_dim_xyz[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    ax.plot(head_pos_xyz[0], head_pos_xyz[1], 'ro')
    src_azim = -180
    src_elev = 20
    src_dist = simulator.distance_to_wall(
        room_dim_xyz,
        head_pos_xyz,
        head_azim=head_azim,
        src_azim=src_azim,
        src_elev=src_elev)
    dhx = 1.4 * np.cos(np.deg2rad(head_azim))
    dhy = 1.4 * np.sin(np.deg2rad(head_azim))
    dx = src_dist * np.cos(np.deg2rad(head_azim + src_azim)) * np.cos(np.deg2rad(src_elev))
    dy = src_dist * np.sin(np.deg2rad(head_azim + src_azim)) * np.cos(np.deg2rad(src_elev))
    ax.plot([head_pos_xyz[0], head_pos_xyz[0] + dhx], [head_pos_xyz[1], head_pos_xyz[1] + dhy], 'r-')
    ax.plot([head_pos_xyz[0], head_pos_xyz[0] + dx], [head_pos_xyz[1], head_pos_xyz[1] + dy])
    ax.set_aspect('equal', 'datalim')
    plt.show()
    
    vector1 = np.array([dx, dy])
    vector2 = np.array([dhx, dhy])
    unit_vector1 = vector1 / np.linalg.norm(vector1)
    unit_vector2 = vector2 / np.linalg.norm(vector2)
    dot_product = np.dot(unit_vector1, unit_vector2)
    angle = np.rad2deg(np.arccos(dot_product))
    print(f'Checked src_azim: {src_azim} = {angle}')
